In [5]:
from bs4 import BeautifulSoup as bs 
import requests 
import pandas as pd 
import numpy as np

HousingData = []
page_index = 1

last_page = None
while True:
    
    url = f'https://www.imovirtual.com/comprar/?page={page_index}'
    page = requests.get(url)

    soup = bs(page.content, 'html.parser')

    if last_page is None:

        pager_next_li = soup.find('li', class_="pager-next")
        previous_li = pager_next_li.find_previous_sibling('li')
        
        last_page = int(previous_li.text)


    lists = soup.find_all('article', class_="offer-item")

    for list in lists:
        link = list.find('a', href=True)["href"]
        price = list.find('li', class_="offer-item-price").get_text(strip=True).split('€')[0]
        metros2 = list.find('strong', class_="visible-xs-block").text.split('m²')[0]
        location = list.find('p', class_="text-nowrap").get_text(strip=True).split(':')[1]
        try:
            rooms = list.find('li', class_="offer-item-rooms").text
        except:
            rooms = np.nan    
        HousingData.append([link, price, metros2, location, rooms])
    
    print(f'{page_index} / {last_page}')
    page_index += 1
    if page_index > last_page: break

1 / 10022
2 / 10022
3 / 10022
4 / 10022
5 / 10022
6 / 10022
7 / 10022
8 / 10022
9 / 10022
10 / 10022
11 / 10022
12 / 10022
13 / 10022
14 / 10022
15 / 10022
16 / 10022
17 / 10022
18 / 10022
19 / 10022
20 / 10022
21 / 10022
22 / 10022
23 / 10022
24 / 10022
25 / 10022
26 / 10022
27 / 10022
28 / 10022
29 / 10022
30 / 10022
31 / 10022
32 / 10022
33 / 10022
34 / 10022
35 / 10022
36 / 10022
37 / 10022
38 / 10022
39 / 10022
40 / 10022
41 / 10022
42 / 10022
43 / 10022
44 / 10022
45 / 10022
46 / 10022
47 / 10022
48 / 10022
49 / 10022
50 / 10022
51 / 10022
52 / 10022
53 / 10022
54 / 10022
55 / 10022
56 / 10022
57 / 10022
58 / 10022
59 / 10022
60 / 10022
61 / 10022
62 / 10022
63 / 10022
64 / 10022
65 / 10022
66 / 10022
67 / 10022
68 / 10022
69 / 10022
70 / 10022
71 / 10022
72 / 10022
73 / 10022
74 / 10022
75 / 10022
76 / 10022
77 / 10022
78 / 10022
79 / 10022
80 / 10022
81 / 10022
82 / 10022
83 / 10022
84 / 10022
85 / 10022
86 / 10022
87 / 10022
88 / 10022
89 / 10022
90 / 10022
91 / 10022
92 / 100

In [14]:
df = pd.DataFrame(HousingData, columns=['link', 'price', 'metros2', 'location', 'rooms'])

df["price"] = df["price"].str.replace(' ', '')
df["price"] = df["price"].str.replace(',', '.')
df["preco_sob_consulta"] = df["price"] == "Preçosobconsulta"
df["price"] = pd.to_numeric(df["price"], errors='coerce').fillna(0)

df["metros2"] = df["metros2"].str.replace(' ', '')
df["metros2"] = df["metros2"].str.replace(',', '.')
df["metros2"] = pd.to_numeric(df["metros2"], errors='coerce').fillna(0)

df["price_meter"] = df["price"] / df["metros2"] 

df['rooms'] = df['rooms'].str.extract('(\d+)').fillna(0).astype(int)

df[['location', 'region']] = df['location'].str.rsplit(',', n=1, expand = True)
import datetime

today = datetime.date.today()
df['last_check'] = today.strftime("%d/%m/%Y")

# df['link'] = df['link'].str.replace('https://www.imovirtual.com/pt/anuncio/', '')
#drop link duplicated
df.drop_duplicates(subset='link', keep='first', inplace=True)

In [21]:

for i in df.loc[i:].index:
    url = df['link'].loc[i]
    print(url)
    page = requests.get(url)
    soup = bs(page.content, 'html.parser')
    try:
        df.at[i,'address'] = soup.find('a',   {'aria-label': 'Endereço'}).get_text(strip=True).split(',')[0]
    except:
        df.at[i,'address'] = np.nan

    dict_labels = {
        'usable_area': 'Área útil (m²)',
        'gross_area' : 'Área bruta (m²)',
        'empreendimento' : 'Empreendimento',
        'typology'  : 'Tipologia',
        'build_year' : 'Ano de construção',
        'num_bathrooms' : 'Casas de Banho',
        'energetic_certification' : 'Certificado Energético',
        'condition' : 'Condição',
    }

    for label in dict_labels.keys():
        if label not in df.columns:
            df[label] = np.nan
        try:
            df.at[i,label] = soup.find('div', {'aria-label': dict_labels[label]}).get_text(strip=True).split(':')[1]
        except:
            df.at[i,label] = np.nan
    
    try:
        df.at[i,'description'] = soup.find('div', {'data-cy' : 'adPageAdDescription'}).get_text(strip=True)
    except:
        print('No description')
    try:
        df.at[i,'other_features'] =  ', '.join([li.get_text(strip=True) for li in soup.find_all('li', {'data-cy' : "ad.ad-features.uncategorized-list.item"})])
    except:
        print('No other_features!')

https://www.imovirtual.com/pt/anuncio/apartamento-t3-a-avenida-do-bessa-ID1czY0.html#d22d5dbf34
https://www.imovirtual.com/pt/anuncio/t1-mezanino-mobilado-e-equipado-com-varanda-cedofeita-ID1czXW.html#d22d5dbf34
https://www.imovirtual.com/pt/anuncio/apartamento-t2-com-terraco-e-estacionamento-no-seixal-ID1czXC.html#d22d5dbf34
https://www.imovirtual.com/pt/anuncio/apartamento-t2-com-terraco-e-estacionamento-no-seixal-ID1czXA.html#d22d5dbf34
https://www.imovirtual.com/pt/anuncio/apartamento-t2-com-terraco-e-estacionamento-no-seixal-ID1czXy.html#d22d5dbf34
https://www.imovirtual.com/pt/anuncio/apartamento-t2-com-terraco-e-estacionamento-no-seixal-ID1czXv.html#d22d5dbf34
https://www.imovirtual.com/pt/anuncio/apartamento-t2-com-terraco-e-estacionamento-no-seixal-ID1czXw.html#d22d5dbf34
https://www.imovirtual.com/pt/anuncio/apartamento-t2-com-terraco-e-estacionamento-no-seixal-ID1czXx.html#d22d5dbf34
https://www.imovirtual.com/pt/anuncio/apartamento-t2-com-terraco-e-estacionamento-no-seixal-

KeyboardInterrupt: 

In [ ]:
df.to_csv('imovirtual_portugal.csv', index=False)

In [23]:
url = 'https://www.imovirtual.com/pt/anuncio/' + df['link'].iloc[0]
page = requests.get(url)
soup = bs(page.content, 'html.parser')

In [49]:
url

'https://www.imovirtual.com/pt/anuncio/apartamento-t2-rua-do-brasil-e-rua-sanches-da-gama-coimbra-ID1cGV9.html#221f689ca1'

In [35]:
# df['link'].iloc[0]
address = soup.find('a', {'aria-label': 'Endereço'}).get_text(strip=True).split(',')[0]
usable_area = soup.find('div', {'aria-label': 'Área útil (m²)'}).get_text(strip=True).split(':')[1]
gross_area = soup.find('div', {'aria-label': 'Área bruta (m²)'}).get_text(strip=True).split(':')[1]
empreendimento = soup.find('div', {'aria-label': 'Empreendimento'}).get_text(strip=True).split(':')[1]
typology = soup.find('div', {'aria-label': 'Tipologia'}).get_text(strip=True).split(':')[1]
build_year = soup.find('div', {'aria-label': 'Ano de construção'}).get_text(strip=True).split(':')[1]
num_bathrooms = soup.find('div', {'aria-label': 'Casas de Banho'}).get_text(strip=True).split(':')[1]
energetic_certification = soup.find('div', {'aria-label': 'Certificado Energético'}).get_text(strip=True).split(':')[1]
condition = soup.find('div', {'aria-label': 'Condição'}).get_text(strip=True).split(':')[1]

description = soup.find('div', {'data-cy' : 'adPageAdDescription'}).get_text(strip=True)





'1990'

In [47]:
description = soup.find('div', {'data-cy' : 'adPageAdDescription'}).get_text(strip=True)

In [72]:
features_text = ', '.join([li.get_text(strip=True) for li in soup.find_all('li', {'data-cy' : "ad.ad-features.uncategorized-list.item"})])

print(features_text)

Negociável, Video Porteiro, Lareira, Fibra Óptica, Gás Canalizado, Vista de cidade, Box (2 carros), Varanda, Arrecadação, Elevador


[<ul class="css-10vi61t e1vdhyh80" data-cy="ad.ad-features.uncategorized-list" role="list"><style data-emotion="css s9zf5i">.css-s9zf5i{width:50%;font-size:15px;line-height:25px;text-transform:capitalize;}@media (min-width:992px){.css-s9zf5i{width:calc(100% / 3);}}</style><li class="css-s9zf5i e1vdhyh81" data-cy="ad.ad-features.uncategorized-list.item">Negociável</li><li class="css-s9zf5i e1vdhyh81" data-cy="ad.ad-features.uncategorized-list.item">Video Porteiro</li><li class="css-s9zf5i e1vdhyh81" data-cy="ad.ad-features.uncategorized-list.item">Lareira</li><li class="css-s9zf5i e1vdhyh81" data-cy="ad.ad-features.uncategorized-list.item">Fibra Óptica</li><li class="css-s9zf5i e1vdhyh81" data-cy="ad.ad-features.uncategorized-list.item">Gás Canalizado</li><li class="css-s9zf5i e1vdhyh81" data-cy="ad.ad-features.uncategorized-list.item">Vista de cidade</li><li class="css-s9zf5i e1vdhyh81" data-cy="ad.ad-features.uncategorized-list.item">Box (2 carros)</li><li class="css-s9zf5i e1vdhyh81"

In [65]:
features_element

[]

In [62]:
features_list[0]

'NegociávelVideo PorteiroLareiraFibra ÓpticaGás CanalizadoVista de cidadeBox (2 carros)VarandaArrecadaçãoElevador'

In [56]:
soup

<!DOCTYPE html>
<html lang="pt"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><link href="https://statics.imovirtual.com" rel="preconnect"/><link href="https://statics.imovirtual.com/static/imovirtualpt/naspersclassifieds-regional/verticalsre-atlas-web-imovirtualpt/static/img/favicon.ico?v=4" rel="icon"/><link href="https://statics.imovirtual.com/static/imovirtualpt/naspersclassifieds-regional/verticalsre-atlas-web-imovirtualpt/static/img/app-icon.png" rel="apple-touch-icon"/><link href="https://statics.imovirtual.com/static/imovirtualpt/naspersclassifieds-regional/verticalsre-atlas-web-imovirtualpt/static/img/app-icon.png" rel="android-touch-icon"/><meta content="425781864171729" property="fb:app_id"/><meta content="Imovirtual: De T0 a T-Tudo! Procura casa para arrendar? Tem apartamento a venda? Pesquise classificados de imobiliárias ou particulares em Portugal: Algarve, Lisboa, Porto, ..." property="og:description"/><meta content="website" property=

In [55]:
features_element = soup.find('div', {'data-cy': 'ad.ad-features.uncategorized-list'})
features_element

In [54]:
features_element

In [53]:
features_text

NameError: name 'features_text' is not defined

In [42]:
len(description)

2739

In [15]:
locations = set(df.location)

{'Almalaguês',
 'Antuzede e Vil de Matos',
 'Assafarge e Antanhol',
 'Brasfemes',
 'Ceira',
 'Cernache',
 'Coimbra (Sé Nova, Santa Cruz, Almedina e São Bartolomeu)',
 'Eiras e São Paulo de Frades',
 'Santa Clara e Castelo Viegas',
 'Santo António dos Olivais',
 'Souselas e Botão',
 'São João do Campo',
 'São Martinho de Árvore e Lamarosa',
 'São Martinho do Bispo e Ribeira de Frades',
 'São Silvestre',
 'Taveiro, Ameal e Arzila',
 'Torres do Mondego',
 'Trouxemil e Torre de Vilela'}

In [ ]:
#"desejado" em ingles é "desire
location_filter = ["Santa Clara e Castelo Viegas", "Santo António dos Olivais"]

In [26]:
# link start with "https://www.imovirtual.com/pt/anuncio/moradia"
df_aux = df.where(df['link'].str.startswith("https://www.imovirtual.com/pt/anuncio/moradia" or "https://www.imovirtual.com/pt/anuncio/apartamento")).dropna()


In [88]:
df[(df["price"] > 140_000) & (df["price"] < 200_000)& (df["rooms"] > 2)].sort_values(by="price_meter", ascending=False)[140:150]

,link,price,metros2,location,rooms,price_meter,region
1734,https://www.imovirtual.com/pt/anuncio/andar-de...,148000.0,142.4,Santa Clara e Castelo Viegas,3,1039.325843,Coimbra
4290,https://www.imovirtual.com/pt/anuncio/moradia-...,160000.0,160.0,Santo António dos Olivais,6,1000.000000,Coimbra
4194,https://www.imovirtual.com/pt/anuncio/moradia-...,160000.0,160.0,Santo António dos Olivais,6,1000.000000,Coimbra
4122,https://www.imovirtual.com/pt/anuncio/moradia-...,160000.0,160.0,Santo António dos Olivais,6,1000.000000,Coimbra
4530,https://www.imovirtual.com/pt/anuncio/moradia-...,160000.0,160.0,Santo António dos Olivais,6,1000.000000,Coimbra
4554,https://www.imovirtual.com/pt/anuncio/moradia-...,160000.0,160.0,Santo António dos Olivais,6,1000.000000,Coimbra
4506,https://www.imovirtual.com/pt/anuncio/moradia-...,160000.0,160.0,Santo António dos Olivais,6,1000.000000,Coimbra
4146,https://www.imovirtual.com/pt/anuncio/moradia-...,160000.0,160.0,Santo António dos Olivais,6,1000.000000,Coimbra
4578,https://www.imovirtual.com/pt/anuncio/moradia-...,160000.0,160.0,Santo António dos Olivais,6,1000.000000,Coimbra
4482,https://www.imovirtual.com/pt/anuncio/moradia-...,160000.0,160.0,Santo António dos Olivais,6,1000.000000,Coimbra


In [89]:
df.loc[4290].link

'https://www.imovirtual.com/pt/anuncio/moradia-t6-para-venda-ID1b3os.html#221f689ca1'